In [45]:
import numpy as np
import pandas as pd

from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NIMBUS_EA import NIMBUS_RVEA, NIMBUS_NSGAIII
import plotly.express as ex
from pygmo import fast_non_dominated_sorting as nds
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline.offline import plot as plothtml

In [4]:
zdt1_pareto = np.asarray([[0.0000000333,0.9998173820],
[0.0036524605,0.9395644103],
[0.0109353082,0.8954279758],
[0.0201661797,0.8579923253],
[0.0307984775,0.8245050499],
[0.0425521730,0.7937182195],
[0.0552536936,0.7649389578],
[0.0687834600,0.7377339899],
[0.0830533346,0.7118102455],
[0.0979952414,0.6869580836],
[0.1135547782,0.6630211013],
[0.1296872770,0.6398788023],
[0.1463553419,0.6174358329],
[0.1635270897,0.5956151713],
[0.1811749649,0.5743534743],
[0.1992748910,0.5535978371],
[0.2178056136,0.5333035102],
[0.2367481746,0.5134322508],
[0.2560856025,0.4939509881],
[0.2758025276,0.4748309533],
[0.2958850236,0.4560468554],
[0.3163203619,0.4375763502],
[0.3370968359,0.4193995902],
[0.3582036897,0.4014987972],
[0.3796309819,0.3838579856],
[0.4013694644,0.3664627364],
[0.4234104957,0.3492999957],
[0.4457460837,0.3323578176],
[0.4683687091,0.3156253153],
[0.4912712857,0.2990925270],
[0.5144471844,0.2827502636],
[0.5378901526,0.2665900514],
[0.5615942516,0.2506040755],
[0.5855539020,0.2347850615],
[0.6097637681,0.2191262791],
[0.6342188315,0.2036214270],
[0.6589142602,0.1882646612],
[0.6838455122,0.1730504779],
[0.7090082355,0.1579737323],
[0.7343982546,0.1430296070],
[0.7600116005,0.1282135580],
[0.7858444564,0.1135213164],
[0.8118931476,0.0989488651],
[0.8381541846,0.0844923897],
[0.8646242088,0.0701482867],
[0.8912999852,0.0559131474],
[0.9181784008,0.0417837401],
[0.9452564510,0.0277570000],
[0.9725312450,0.0138300121],
[1.0000000000,0.0000000000],])

In [5]:
problem = test_problem_builder(name="ZDT1")

In [6]:
problem.ideal = np.asarray([0, 0])
problem.nadir = np.asarray([1, 1])

In [ ]:
evolver = NIMBUS_NSGAIII(problem, n_gen_per_iter=100)

In [ ]:
plot, pref = evolver.requests()

In [ ]:
response = np.asarray([[0.5,0.1]])
pref.response = pd.DataFrame(response, columns=pref.content['dimensions_data'].columns)

In [ ]:
plot, pref = evolver.iterate(pref)

In [ ]:
# Getting the solutions non-dominated in the PMS
scaled_data = np.asarray([scalar(evolver.population.fitness, response) for scalar in evolver.scalarization_methods]).T
non_dom_ids = nds(scaled_data)[0][0]


nimbus = plt.scatter(
    ne = dict(col
)
"""nimbus = plt.scatter(
    x=evolver.population.objectives[:, 0],
    y=evolver.population.objectives[:, 1],
    label='NIMBUS-RVEA'
)"""
refp = plt.scatter(x=response[:,0], y=response[:,1], c='r', label='Reference Point')
stom = plt.arrow(x=response[0,0], y=response[0,1], dx=response[0,0]*10, dy=response[0,1]*10, color='g', label='STOM')
asf = plt.arrow(x=response[0,0], y=response[0,1], dx=10, dy=10,color='black',label="ASF")
aug = plt.arrow(x=response[0,0], y=response[0,1], dx=1-response[0,0], dy=1-response[0,1],color='red',label="AUG")
# Completing the other side
plt.arrow(x=response[0,0], y=response[0,1], dx=-response[0,0]*10, dy=-response[0,1]*10, color='g')
plt.arrow(x=response[0,0], y=response[0,1], dx=-10, dy=-10, color='black')
plt.arrow(x=response[0,0], y=response[0,1], dx=-1+response[0,0], dy=-1+response[0,1],color='red')
pareto = plt.plot(zdt1_pareto[:,0], zdt1_pareto[:,1],'-.')
plt.xlim([0,1])
plt.ylim([0,1])
plt.title("ZDT1 problem with NIMBUS inspired EA")
plt.legend([nimbus, refp, stom, asf, pareto], ["NIMBUS-RVEA","Reference Point","STOM", "ASF", "AUG", "Pareto optimal solutions"])
plt.gca().set_aspect('equal', adjustable='box')

In [36]:
def subbplot(evolver, pref, showlegend):
    # Getting the solutions non-dominated in the PMS
    scaled_data = np.asarray([scalar(evolver.population.fitness, response) for scalar in evolver.scalarization_methods]).T
    non_dom_ids = nds(scaled_data)[0][0]
    
    colorlist = ex.colors.qualitative.D3
    
    fig = go.Figure()
    # NIMBUS EA
    nimbus = go.Scatter(x=evolver.population.objectives[non_dom_ids, 0],
                        y=evolver.population.objectives[non_dom_ids, 1],
                        legendgroup='NIMBUS-RVEA',
                        name='NIMBUS-RVEA',
                        mode="markers",
                        marker=dict(size=10, color = colorlist[0]),
                        showlegend=showlegend)
    # WHOLE FRONT
    front = go.Scatter(x=zdt1_pareto[:,0],
                       y=zdt1_pareto[:,1],
                       legendgroup="Pareto optimal solutions",
                       name="Pareto optimal solutions",
                       mode="lines",
                       line=dict(dash="dashdot", color = colorlist[1]),
                       showlegend=showlegend)
    
    ideal = evolver.population.problem.ideal
    nadir = evolver.population.problem.nadir
    
    #AUG: z+l*(n-z)<=i
    l_aug = max((ideal- pref)/(nadir-pref))
    end = pref + l_aug * (nadir - pref)
    aug = go.Scatter(x=[nadir[0], end[0]],
                     y = [nadir[1], end[1]],
                     legendgroup="AUG",
                     name="AUG",
                     mode="lines",
                     line=dict(dash="longdash", color = colorlist[2]),
                     showlegend=showlegend
                    )
    #STOM: z+l*(z-i)>=n
    l_aug = min((nadir- pref)/(pref-ideal))
    end = pref + l_aug * (pref - ideal)
    stom = go.Scatter(x=[ideal[0], end[0]],
                     y = [ideal[1], end[1]],
                     legendgroup="STOM",
                     name="STOM",
                     mode="lines",
                     line=dict(dash="dot", color = colorlist[3]),
                     showlegend=showlegend
                    )
    
    # ideal point
    ide = go.Scatter(x=[ideal[0]],
                     y = [ideal[1]],
                     legendgroup="Ideal point",
                     name="Ideal Point",
                     mode="markers",
                     marker=dict(size=15, color = colorlist[4]),
                     showlegend=showlegend)
    # nadir point
    nad = go.Scatter(x=[nadir[0]],
                     y = [nadir[1]],
                     legendgroup="Nadir point",
                     name="Nadir Point",
                     mode="markers",
                     marker=dict(size=15, color = colorlist[5]),
                     showlegend=showlegend)
    # reference point
    refp = go.Scatter(x=[pref[0]],
                     y = [pref[1]],
                     legendgroup="Reference point",
                     name="Reference Point",
                     mode="markers",
                     marker=dict(size=15, color = colorlist[6]),
                     showlegend=showlegend)
    
    return(nimbus, front, aug, stom, ide, nad, refp)

In [41]:
responses = np.asarray([[0.9,0.2],[0.5,0.5],[0.1,0.3],[0.2,0.5],])
evolver = NIMBUS_NSGAIII(problem, n_gen_per_iter=100)
plot, pref = evolver.requests()
fig = make_subplots(rows=2, cols=2, row_heights=[1,1], column_widths=[1,1])
fig.update_xaxes(range=[0, 1])
fig.update_yaxes(range=[0, 1])
subplots = [[1,1],[1,2],[2,1],[2,2]]
for num, (response, pos) in enumerate(zip(responses, subplots)):
    pref.response = pd.DataFrame([response], columns=pref.content['dimensions_data'].columns)
    plot, pref = evolver.iterate(pref)
    showlegend=False if num<3 else True
    sub_fig = subbplot(evolver, response, showlegend)
    for sub in sub_fig:
        fig.add_trace(sub, row=pos[0],col=pos[1])

In [42]:
fig.update_layout(template="simple_white")
fig.update_xaxes(showline=True, linewidth=1.5, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='black', mirror=True)
fig.update_xaxes(range=[0, 1])
fig.update_yaxes(range=[0, 1])
#fig.update_layout(width=900)
#fig.update_layout(height=800)

"""for subnum in ["", "2","3","4"]:
    fig["layout"]["xaxis"+subnum]["constrain"] = "domain"
    fig["layout"]["yaxis"+subnum]["scaleanchor"] = "x"+subnum
    fig["layout"]["yaxis"+subnum]["scaleratio"] =1
fig.update_layout(legend_orientation='h')"""
#fig.update_layout(showlegend=False)
fig

In [46]:
plothtml(fig)

'temp-plot.html'